In [303]:
import polars as pl
import os
from pathlib import Path
import pandas as pd
import hvplot.polars
import datetime
import matplotlib.pyplot as plt
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

data_path = Path.cwd().parent.parent / "Data"

#Restrictions
timeperiod = [datetime.datetime(2018, 12, 30), datetime.datetime(2021, 12, 31)]
train_period = [datetime.datetime(2019, 1, 1), datetime.datetime(2020, 12, 31)]
trainperiod1 = [datetime.datetime(2019, 1, 1), datetime.datetime(2019, 12, 31)]
trainperiod2 = [datetime.datetime(2020, 1, 1), datetime.datetime(2020, 12, 31)]
volume_r = 0
open_interest_r = 0
delta_r = 0
vega_r = 0
theta_r = 0
gamma_r = 0
midprice_r = 0.3
bid_price_r = 0.0
days_till_exp_r1 = 7
days_till_exp_r2 = 252
moneyness_min = -2.5
moneyness_max = 2.5


In [304]:
firm_stock_data = Path.cwd().parent.parent / "Data/updated_standardization/firm_stock_dataset_tech.parquet"
option_data = Path.cwd().parent.parent / "Data/updated_standardization/data_set_option_tech.parquet"
macro_data = Path.cwd().parent.parent / "Data/updated_standardization/data_set_macro.parquet"

firm_stock_data = pd.read_parquet(firm_stock_data)
option_data = pd.read_parquet(option_data)
macro_data = pd.read_parquet(macro_data)

# List of columns to drop
columns_to_drop = ['divi', 'divo', 'sin']

# Drop columns from datasets if they exist
firm_stock_data = firm_stock_data.drop(columns=columns_to_drop, errors='ignore')
option_data = option_data.drop(columns=columns_to_drop, errors='ignore')
macro_data = macro_data.drop(columns=columns_to_drop, errors='ignore')

firm_train =  firm_stock_data[(firm_stock_data["date"] >= train_period[0]) & (firm_stock_data["date"] <= train_period[1])]
firm_train1 = firm_stock_data[(firm_stock_data["date"] >= trainperiod1[0]) & (firm_stock_data["date"] <= trainperiod1[1])]
firm_train2 = firm_stock_data[(firm_stock_data["date"] >= trainperiod2[0]) & (firm_stock_data["date"] <= trainperiod2[1])]

option_train = option_data[(option_data["date"] >= train_period[0]) & (option_data["date"] <= train_period[1])]
option_train1 = option_data[(option_data["date"] >= trainperiod1[0]) & (option_data["date"] <= trainperiod1[1])]
option_train2 = option_data[(option_data["date"] >= trainperiod2[0]) & (option_data["date"] <= trainperiod2[1])]
                            

macro_train = macro_data[(macro_data["date"] >= train_period[0]) & (macro_data["date"] <= train_period[1])]
macro_train1 = macro_data[(macro_data["date"] >= trainperiod1[0]) & (macro_data["date"] <= trainperiod1[1])]
macro_train2 = macro_data[(macro_data["date"] >= trainperiod2[0]) & (macro_data["date"] <= trainperiod2[1])]

firm_test = firm_stock_data[(firm_stock_data["date"] > train_period[1])]
option_test = option_data[(option_data["date"] > train_period[1])]
macro_test = macro_data[(macro_data["date"] > train_period[1])]

In [305]:
option_test

,cp_flag,Ticker,date,trading_days_till_exp,moneyness,impl_volatility,T,prev_day_iv,prev2_day_iv
75102,C,AAPL,2021-01-04,4,-2.411,0.599475,0.019841,0.605595,0.327228
75103,C,AAPL,2021-01-04,4,-2.265,0.610254,0.019841,0.605595,0.327228
75104,C,AAPL,2021-01-04,4,-2.120,0.570884,0.019841,0.605595,0.327228
75105,C,AAPL,2021-01-04,4,-1.977,0.566679,0.019841,0.605595,0.327228
75106,C,AAPL,2021-01-04,4,-1.835,0.525325,0.019841,0.448299,0.559822
...,...,...,...,...,...,...,...,...,...
428240,P,TSLA,2021-12-30,1,2.062,1.012930,0.007937,0.753005,0.932566
428241,P,TSLA,2021-12-30,1,2.161,1.047433,0.007937,0.753005,0.932566
428242,P,TSLA,2021-12-30,1,2.261,1.088575,0.007937,0.753005,0.932566
428243,P,TSLA,2021-12-30,1,2.360,1.129544,0.007937,0.753005,0.932566


# Standardize macro data

In [306]:
import pandas as pd

def standardize(df):
    """Standardize the numeric columns, excluding the 'date' column."""
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
    return df

# Assuming your DataFrame is called 'macro_train'
data = macro_train.copy()

# Exclude the 'date' column from the columns to be standardized
columns_to_standardize = data.columns.difference(['date'])

# Standardize the entire DataFrame (except 'date')
standardized_data = standardize(data[columns_to_standardize])

# Combine standardized numeric data with the 'date' column
standardized_data_macro = pd.concat([data[['date']], standardized_data], axis=1)

# Display the standardized DataFrame
standardized_data_macro


/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_9160/740111496.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()


,date,10Y_RIR,1Y_bond,2Y_bond,CLOSE_vix,FF_rate,HIGH_vix,LOW_vix,OPEN_vix,gold_price,reces_indi,spread_vix
3,2019-01-02,1.562822,1.371916,1.469739,0.476238,1.124610,0.475155,0.662651,0.674867,-1.353864,-0.709827,-0.133918
4,2019-01-03,1.562822,1.371916,1.469739,0.071348,1.124610,0.457626,0.198016,0.656238,-1.349397,-0.709394,1.065358
5,2019-01-04,1.426211,1.310038,1.377262,0.336815,1.124610,0.300487,0.331346,0.252127,-1.344931,-0.708961,0.153294
8,2019-01-07,1.576483,1.355593,1.441962,-0.067370,1.124610,0.014381,0.038020,0.124589,-1.340465,-0.708528,-0.052797
9,2019-01-08,1.617466,1.392399,1.488480,-0.023713,1.124610,-0.045720,0.059838,0.080165,-1.327066,-0.707228,-0.322470
...,...,...,...,...,...,...,...,...,...,...,...,...
725,2020-12-24,-0.759553,-1.214549,-1.201615,-0.183556,-1.160795,-0.265465,-0.221367,-0.171329,1.333629,0.283189,-0.329047
729,2020-12-28,-0.786875,-1.215936,-1.205631,-0.330020,-1.160795,-0.311167,-0.284395,-0.223634,1.348571,0.281024,-0.318085
730,2020-12-29,-0.786875,-1.209428,-1.202285,-0.123703,-1.160795,-0.233536,-0.259346,-0.368369,1.408338,0.272360,-0.114186
731,2020-12-30,-0.786875,-1.206761,-1.201057,-0.023713,-1.160795,-0.092048,-0.049250,-0.162015,1.423280,0.270195,-0.188730


In [307]:
import pandas as pd

def standardize(df):
    """Standardize the numeric columns, excluding the 'date' column."""
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
    return df

# Assuming your DataFrame is called 'macro_train'
data = firm_train.copy()

# Exclude the 'date' column from the columns to be standardized
columns_to_standardize = data.columns.difference(['Ticker', 'date'])

# Standardize the entire DataFrame (except 'date')
standardized_data = standardize(data[columns_to_standardize])

# Combine standardized numeric data with the 'date' column
standardized_data_firm = pd.concat([data[['Ticker' ,'date']], standardized_data], axis=1)

# Display the standardized DataFrame
standardized_data_firm


/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_9160/1255623922.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()


,Ticker,date,5_day_rolling_return_stock,ASK,ASKHI,BID,BIDLO,PRC,PRC_actual,RET,...,spread_stock,std_dolvol,std_turn,stdacc,stdcf,tang,tb,turn,vol_stock,zerotrade
0,AAPL,2019-01-02,-0.286276,-0.568980,-0.569260,-0.569010,-0.567166,-0.568991,-1.119896,-0.080584,...,-0.125902,0.384411,-0.383392,-1.637978,-1.397221,-2.269561,-0.150957,-0.464434,0.737871,-0.186964
1,AAPL,2019-01-03,0.513255,-0.566836,-0.568214,-0.566853,-0.564699,-0.567087,-1.114221,0.234178,...,-0.131505,0.340969,-0.384122,-1.637978,-1.397221,-2.269561,-0.150957,-0.463254,0.462326,-0.179913
2,AAPL,2019-01-04,0.974270,-0.566849,-0.568849,-0.566878,-0.567575,-0.566860,-1.113545,-0.058953,...,-0.125914,0.297528,-0.384853,-1.637978,-1.397221,-2.269561,-0.150957,-0.462073,0.540321,-0.172861
3,AAPL,2019-01-07,-1.739500,-0.586819,-0.585200,-0.586846,-0.583208,-0.586694,-1.172660,-3.523569,...,-0.131510,0.167203,-0.387044,-1.637978,-1.397221,-2.269561,-0.150957,-0.458530,2.593612,-0.151707
4,AAPL,2019-01-08,-1.028140,-0.579040,-0.581676,-0.579063,-0.580907,-0.579040,-1.149848,1.369838,...,-0.131516,0.123762,-0.387774,-1.637978,-1.397221,-2.269561,-0.150957,-0.457350,1.354594,-0.144656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,TSLA,2020-12-24,-0.047648,0.041302,0.042610,0.041255,0.020426,0.041417,1.501621,-0.601955,...,0.030951,2.181198,0.928123,1.677337,0.982961,-0.057908,-0.411748,1.730563,1.095747,-1.103519
5800,TSLA,2020-12-28,0.366524,0.048437,0.044627,0.048381,0.031086,0.048528,1.529882,0.204706,...,0.036559,2.620721,1.011200,1.677337,0.982961,-0.057908,-0.411748,1.765113,0.388826,-1.097611
5801,TSLA,2020-12-29,-0.083321,0.068356,0.062795,0.068501,0.054645,0.068437,1.609004,0.742412,...,-0.047459,2.730602,1.031969,1.677337,0.982961,-0.057908,-0.411748,1.773751,0.001428,-1.096134
5802,TSLA,2020-12-30,-0.941110,0.070865,0.081860,0.070998,0.079954,0.070858,1.618625,0.001571,...,-0.041851,2.840483,1.052738,1.677337,0.982961,-0.057908,-0.411748,1.782388,0.357568,-1.094657


# Calls standardization

In [308]:
option_train_c  = option_train.copy()

# Apply filter of moneyness
option_train_c = option_train_c[option_train_c["cp_flag"] == "C"]
option_train_c = option_train_c[(option_train_c["moneyness"] >= -0.5) & (option_train_c["moneyness"] <= 2)]

import pandas as pd

def standardize(df):
    """Standardize the numeric columns, excluding the 'date' column."""
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
    return df

# Assuming your DataFrame is called 'macro_train'
data = option_train_c.copy()

# Exclude the 'date' column from the columns to be standardized
columns_to_standardize = data.columns.difference(['Ticker', 'date', 'cp_flag'])

# Standardize the entire DataFrame (except 'date')
standardized_data = standardize(data[columns_to_standardize])

# Combine standardized numeric data with the 'date' column
standardized_data_opt_c = pd.concat([data[['Ticker' ,'date', 'cp_flag']], standardized_data], axis=1)

# Display the standardized DataFrame
standardized_data_opt_c


/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_9160/3018573154.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()


,Ticker,date,cp_flag,T,impl_volatility,moneyness,prev2_day_iv,prev_day_iv,trading_days_till_exp
64436,AAPL,2019-01-02,C,-0.854054,-0.592091,-1.651490,-1.519873,-1.542073,-0.854054
64437,AAPL,2019-01-02,C,-0.854054,-0.579443,-1.062773,-1.519873,-0.759052,-0.854054
64438,AAPL,2019-01-02,C,-0.854054,-0.586154,-0.482466,-1.519873,-0.716917,-0.854054
64439,AAPL,2019-01-02,C,-0.854054,-0.577218,0.088029,-1.519873,-0.693295,-0.854054
64440,AAPL,2019-01-02,C,-0.854054,-0.562136,0.650114,-1.519873,-0.674080,-0.854054
...,...,...,...,...,...,...,...,...,...
234752,TSLA,2020-12-31,C,1.331335,0.167539,1.060814,-0.140819,0.024429,1.331335
234753,TSLA,2020-12-31,C,1.331335,0.185279,1.156130,-0.140819,0.024429,1.331335
234754,TSLA,2020-12-31,C,1.331335,0.212952,1.345361,-0.071801,0.110670,1.331335
234755,TSLA,2020-12-31,C,1.331335,0.237258,1.533190,-0.037014,0.158024,1.331335


# Puts standardization

In [309]:
option_train_p  = option_train.copy()
option_train_p = option_train_p[option_train_p["cp_flag"] == "P"]
option_train_p = option_train_p[(option_train_p["moneyness"] >= -2) & (option_train_p["moneyness"] <= 0.5)]

import pandas as pd

def standardize(df):
    """Standardize the numeric columns, excluding the 'date' column."""
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
    return df

# Assuming your DataFrame is called 'macro_train'
data = option_train_p.copy()

# Exclude the 'date' column from the columns to be standardized
columns_to_standardize = data.columns.difference(['Ticker', 'date', 'cp_flag'])

# Standardize the entire DataFrame (except 'date')
standardized_data = standardize(data[columns_to_standardize])

# Combine standardized numeric data with the 'date' column
standardized_data_opt_p = pd.concat([data[['Ticker' ,'date', 'cp_flag']], standardized_data], axis=1)

# Display the standardized DataFrame
standardized_data_opt_p


/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_9160/1112321097.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()


,Ticker,date,cp_flag,T,impl_volatility,moneyness,prev2_day_iv,prev_day_iv,trading_days_till_exp
251194,AAPL,2019-01-02,P,-0.862489,-0.118447,-1.599060,-1.565542,-0.215265,-0.862489
251195,AAPL,2019-01-02,P,-0.862489,-0.161328,-1.348787,-1.565542,-0.243882,-0.862489
251196,AAPL,2019-01-02,P,-0.862489,-0.202412,-1.099920,-1.565542,-0.272954,-0.862489
251197,AAPL,2019-01-02,P,-0.862489,-0.246881,-0.853865,-1.565542,-0.301999,-0.862489
251198,AAPL,2019-01-02,P,-0.862489,-0.345423,-0.242243,-1.565542,-0.349271,-0.862489
...,...,...,...,...,...,...,...,...,...
413055,TSLA,2020-12-31,P,1.327470,-0.078858,1.347974,-0.388218,-0.077136,1.327470
413056,TSLA,2020-12-31,P,1.327470,-0.079386,1.401403,-0.388671,-0.113290,1.327470
413057,TSLA,2020-12-31,P,1.327470,-0.080922,1.454832,-0.388671,-0.113290,1.327470
413058,TSLA,2020-12-31,P,1.327470,-0.080638,1.561690,-0.385860,-0.145900,1.327470


# Merge all the testdata

In [310]:
# Apply filter of moneyness
option_train_f = option_test.copy()

option_train_data_c = option_test[option_train_f["cp_flag"] == "C"]
option_train_data_c = option_train_data_c[(option_train_data_c["moneyness"] >= -0.5) & (option_train_data_c["moneyness"] <= 2)]

option_train_data_p = option_train_f[option_train_f["cp_flag"] == "P"]
option_train_data_p = option_train_data_p[(option_train_data_p["moneyness"] <= 0.5) & (option_train_data_p["moneyness"] >= -2)]

options_total = pd.concat([option_train_data_c, option_train_data_p], axis=0)
# Merge with the option data & macro data
merge_data_t = pd.merge(options_total, firm_test, on=['date', 'Ticker'], how='left')

# Merge the data but now with the macro data
merge_data_total_t = pd.merge(merge_data_t, macro_test, on='date', how='left')

merge_data_total_t

,cp_flag,Ticker,date,trading_days_till_exp,moneyness,impl_volatility,T,prev_day_iv,prev2_day_iv,BIDLO,...,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,HIGH_vix,LOW_vix,CLOSE_vix,spread_vix
0,C,AAPL,2021-01-04,4,-0.475,0.433412,0.019841,0.337114,0.325650,133.39999,...,672.012903,3.036452,0.93,0.1074,0.1400,21.29,22.49,20.57,21.20,1.92
1,C,AAPL,2021-01-04,4,-0.345,0.428775,0.019841,0.332640,0.312515,133.39999,...,672.012903,3.036452,0.93,0.1074,0.1400,21.29,22.49,20.57,21.20,1.92
2,C,AAPL,2021-01-04,4,-0.216,0.426358,0.019841,0.330864,0.308907,133.39999,...,672.012903,3.036452,0.93,0.1074,0.1400,21.29,22.49,20.57,21.20,1.92
3,C,AAPL,2021-01-04,4,-0.088,0.422649,0.019841,0.329431,0.306933,133.39999,...,672.012903,3.036452,0.93,0.1074,0.1400,21.29,22.49,20.57,21.20,1.92
4,C,AAPL,2021-01-04,4,0.039,0.421479,0.019841,0.328318,0.305940,133.39999,...,672.012903,3.036452,0.93,0.1074,0.1400,21.29,22.49,20.57,21.20,1.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71690,P,TSLA,2021-12-30,1,-0.027,0.484270,0.007937,0.520039,0.592762,1078.42004,...,632.874194,-0.206774,1.55,0.4878,0.8182,14.36,14.60,12.56,12.85,2.04
71691,P,TSLA,2021-12-30,1,0.081,0.503165,0.007937,0.520221,0.600908,1078.42004,...,632.874194,-0.206774,1.55,0.4878,0.8182,14.36,14.60,12.56,12.85,2.04
71692,P,TSLA,2021-12-30,1,0.190,0.536047,0.007937,0.525230,0.609799,1078.42004,...,632.874194,-0.206774,1.55,0.4878,0.8182,14.36,14.60,12.56,12.85,2.04
71693,P,TSLA,2021-12-30,1,0.298,0.550971,0.007937,0.529985,0.620276,1078.42004,...,632.874194,-0.206774,1.55,0.4878,0.8182,14.36,14.60,12.56,12.85,2.04


In [311]:
# Check for NaN values column-wise
nan_columns = merge_data_total_t.isnull().sum()
print("Number of NaN values in each column:")
print(nan_columns)

# Add columns with NaN values to a list
nan_columns_list = nan_columns[nan_columns > 0].index.tolist()
print("Columns with NaN values:")
print(nan_columns_list)

Number of NaN values in each column:
cp_flag                  0
Ticker                   0
date                     0
trading_days_till_exp    0
moneyness                0
                        ..
OPEN_vix                 0
HIGH_vix                 0
LOW_vix                  0
CLOSE_vix                0
spread_vix               0
Length: 126, dtype: int64
Columns with NaN values:
[]


In [312]:
# Testset save to parquet

data_set_standardized_test = Path.cwd().parent.parent / "Data/updated_standardization/data_set_test_tech_nonscaled.parquet"

save_parquet = True
if save_parquet or not os.path.exists(data_set_standardized_test):
    merge_data_total_t.to_parquet(data_set_standardized_test)

# Merge all the standardized data with each other

In [313]:
# # Merge the option data

# # Concatenate the two DataFrames vertically
# combined_data = pd.concat([standardized_data_opt_p, standardized_data_opt_c], axis=0)

# # Reset the index if necessary
# combined_data.reset_index(drop=True, inplace=True)

# # Display the combined DataFrame
# (combined_data)

In [314]:
# Merge with the firm data & macro data
merge_data_p = pd.merge(standardized_data_opt_p, standardized_data_firm, on=['date', 'Ticker'], how='left')
merge_data_c = pd.merge(standardized_data_opt_c, standardized_data_firm, on=['date', 'Ticker'], how='left')

# Merge the data but now with the macro data
merge_data_total_p = pd.merge(merge_data_p, standardized_data_macro, on='date', how='left')
merge_data_total_c = pd.merge(merge_data_c, standardized_data_macro, on='date', how='left')

merge_data_total_p

,Ticker,date,cp_flag,T,impl_volatility,moneyness,prev2_day_iv,prev_day_iv,trading_days_till_exp,5_day_rolling_return_stock,...,1Y_bond,2Y_bond,CLOSE_vix,FF_rate,HIGH_vix,LOW_vix,OPEN_vix,gold_price,reces_indi,spread_vix
0,AAPL,2019-01-02,P,-0.862489,-0.118447,-1.599060,-1.565542,-0.215265,-0.862489,-0.286276,...,1.371916,1.469739,0.476238,1.124610,0.475155,0.662651,0.674867,-1.353864,-0.709827,-0.133918
1,AAPL,2019-01-02,P,-0.862489,-0.161328,-1.348787,-1.565542,-0.243882,-0.862489,-0.286276,...,1.371916,1.469739,0.476238,1.124610,0.475155,0.662651,0.674867,-1.353864,-0.709827,-0.133918
2,AAPL,2019-01-02,P,-0.862489,-0.202412,-1.099920,-1.565542,-0.272954,-0.862489,-0.286276,...,1.371916,1.469739,0.476238,1.124610,0.475155,0.662651,0.674867,-1.353864,-0.709827,-0.133918
3,AAPL,2019-01-02,P,-0.862489,-0.246881,-0.853865,-1.565542,-0.301999,-0.862489,-0.286276,...,1.371916,1.469739,0.476238,1.124610,0.475155,0.662651,0.674867,-1.353864,-0.709827,-0.133918
4,AAPL,2019-01-02,P,-0.862489,-0.345423,-0.242243,-1.565542,-0.349271,-0.862489,-0.286276,...,1.371916,1.469739,0.476238,1.124610,0.475155,0.662651,0.674867,-1.353864,-0.709827,-0.133918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72959,TSLA,2020-12-31,P,1.327470,-0.078858,1.347974,-0.388218,-0.077136,1.327470,0.168781,...,-1.213802,-1.207416,-0.034979,-1.160795,-0.145889,0.015395,-0.057404,1.438222,0.268029,-0.552678
72960,TSLA,2020-12-31,P,1.327470,-0.079386,1.401403,-0.388671,-0.113290,1.327470,0.168781,...,-1.213802,-1.207416,-0.034979,-1.160795,-0.145889,0.015395,-0.057404,1.438222,0.268029,-0.552678
72961,TSLA,2020-12-31,P,1.327470,-0.080922,1.454832,-0.388671,-0.113290,1.327470,0.168781,...,-1.213802,-1.207416,-0.034979,-1.160795,-0.145889,0.015395,-0.057404,1.438222,0.268029,-0.552678
72962,TSLA,2020-12-31,P,1.327470,-0.080638,1.561690,-0.385860,-0.145900,1.327470,0.168781,...,-1.213802,-1.207416,-0.034979,-1.160795,-0.145889,0.015395,-0.057404,1.438222,0.268029,-0.552678


In [315]:
# Check for NaN values column-wise
nan_columns = merge_data_total_p.isnull().sum()
print("Number of NaN values in each column:")
print(nan_columns)

# Add columns with NaN values to a list
nan_columns_list = nan_columns[nan_columns > 0].index.tolist()
print("Columns with NaN values:")
print(nan_columns_list)

# Drop columns with NaN values
merge_data_total_p = merge_data_total_p.drop(columns=nan_columns_list)

Number of NaN values in each column:
Ticker             0
date               0
cp_flag            0
T                  0
impl_volatility    0
                  ..
LOW_vix            0
OPEN_vix           0
gold_price         0
reces_indi         0
spread_vix         0
Length: 126, dtype: int64
Columns with NaN values:
[]


In [316]:
# Save the parquet file

data_set_standardized_tot_p = Path.cwd().parent.parent / "Data/updated_standardization/data_set_train_val_tech_scaled_p.parquet"
data_set_standardized_tot_c = Path.cwd().parent.parent / "Data/updated_standardization/data_set_train_val_tech_scaled_c.parquet"

save_parquet = True
if save_parquet or not os.path.exists(data_set_standardized_tot_p):
    merge_data_total_p.to_parquet(data_set_standardized_tot_p)

save_parquet = True
if save_parquet or not os.path.exists(data_set_standardized_tot_c):
    merge_data_total_c.to_parquet(data_set_standardized_tot_c)

# Merge nonscaled traingsset tech

In [317]:
# Merge with the option data & macro data
option_train_f  = option_train.copy()

# Apply filter of moneyness
option_train_data_c = option_train_f[option_train_f["cp_flag"] == "C"]
option_train_data_c = option_train_data_c[(option_train_data_c["moneyness"] >= -0.5) & (option_train_data_c["moneyness"] <= 2)]

option_train_data_p = option_train_f[option_train_f["cp_flag"] == "P"]
option_train_data_p = option_train_data_p[(option_train_data_p["moneyness"] <= 0.5) & (option_train_data_p["moneyness"] >= -2)]

# Concatenate the two DataFrames vertically
optiondata_train = pd.concat([option_train_data_p, option_train_data_c], axis=0)

# Reset the index if necessary
optiondata_train.reset_index(drop=True, inplace=True)

optiondata_train

,cp_flag,Ticker,date,trading_days_till_exp,moneyness,impl_volatility,T,prev_day_iv,prev2_day_iv
0,P,AAPL,2019-01-02,2,-1.860,0.558181,0.011905,0.505761,0.000000
1,P,AAPL,2019-01-02,2,-1.682,0.541292,0.011905,0.495051,0.000000
2,P,AAPL,2019-01-02,2,-1.505,0.525111,0.011905,0.484170,0.000000
3,P,AAPL,2019-01-02,2,-1.330,0.507597,0.011905,0.473299,0.000000
4,P,AAPL,2019-01-02,2,-0.895,0.468786,0.011905,0.455607,0.000000
...,...,...,...,...,...,...,...,...,...
145306,C,TSLA,2020-12-31,5,1.467,0.623205,0.023810,0.554273,0.486818
145307,C,TSLA,2020-12-31,5,1.535,0.629719,0.023810,0.554273,0.486818
145308,C,TSLA,2020-12-31,5,1.670,0.639880,0.023810,0.584788,0.511182
145309,C,TSLA,2020-12-31,5,1.804,0.648805,0.023810,0.601543,0.523462


In [318]:
option_train_data_p

,cp_flag,Ticker,date,trading_days_till_exp,moneyness,impl_volatility,T,prev_day_iv,prev2_day_iv
251194,P,AAPL,2019-01-02,2,-1.860,0.558181,0.011905,0.505761,0.000000
251195,P,AAPL,2019-01-02,2,-1.682,0.541292,0.011905,0.495051,0.000000
251196,P,AAPL,2019-01-02,2,-1.505,0.525111,0.011905,0.484170,0.000000
251197,P,AAPL,2019-01-02,2,-1.330,0.507597,0.011905,0.473299,0.000000
251198,P,AAPL,2019-01-02,2,-0.895,0.468786,0.011905,0.455607,0.000000
...,...,...,...,...,...,...,...,...,...
413055,P,TSLA,2020-12-31,5,0.236,0.573773,0.023810,0.557458,0.432122
413056,P,TSLA,2020-12-31,5,0.274,0.573565,0.023810,0.543927,0.431956
413057,P,TSLA,2020-12-31,5,0.312,0.572960,0.023810,0.543927,0.431956
413058,P,TSLA,2020-12-31,5,0.388,0.573072,0.023810,0.531722,0.432987


In [319]:
merge_data_train = pd.merge(optiondata_train, firm_train, on=['date', 'Ticker'], how='left')

# Merge the data but now with the macro data
merge_data_total_train = pd.merge(merge_data_train, macro_train, on='date', how='left')

In [320]:
# Check for NaN values column-wise
nan_columns = merge_data_total_train.isnull().sum()
print("Number of NaN values in each column:")
print(nan_columns)

# Add columns with NaN values to a list
nan_columns_list = nan_columns[nan_columns > 0].index.tolist()
print("Columns with NaN values:")
print(nan_columns_list)

# Drop columns with NaN values
merge_data_total_train = merge_data_total_train.drop(columns=nan_columns_list)

Number of NaN values in each column:
cp_flag                  0
Ticker                   0
date                     0
trading_days_till_exp    0
moneyness                0
                        ..
OPEN_vix                 0
HIGH_vix                 0
LOW_vix                  0
CLOSE_vix                0
spread_vix               0
Length: 126, dtype: int64
Columns with NaN values:
[]


In [321]:
# Save the parquet file

data_set_train_nonscaled = Path.cwd().parent.parent / "Data/updated_standardization/data_set_train_tech_nonscaled.parquet"

save_parquet = True
if save_parquet or not os.path.exists(data_set_train_nonscaled):
    merge_data_total_train.to_parquet(data_set_train_nonscaled)

# Merge scaled dataset test

In [322]:
# # Standardize micro data

# import pandas as pd

# def standardize(df):
#     """Standardize the numeric columns, excluding the 'date' column."""
#     numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
#     df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
#     return df

# # Assuming your DataFrame is called 'macro_train'
# data = macro_test.copy()

# # Exclude the 'date' column from the columns to be standardized
# columns_to_standardize = data.columns.difference(['date'])

# # Standardize the entire DataFrame (except 'date')
# standardized_data = standardize(data[columns_to_standardize])

# # Combine standardized numeric data with the 'date' column
# standardized_data_macro_t = pd.concat([data[['date']], standardized_data], axis=1)

# # Display the standardized DataFrame
# standardized_data_macro_t


In [323]:
# import pandas as pd

# def standardize(df):
#     """Standardize the numeric columns, excluding the 'date' column."""
#     numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
#     df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
#     return df

# # Assuming your DataFrame is called 'macro_train'
# data = firm_test.copy()

# # Exclude the 'date' column from the columns to be standardized
# columns_to_standardize = data.columns.difference(['Ticker', 'date'])

# # Standardize the entire DataFrame (except 'date')
# standardized_data = standardize(data[columns_to_standardize])

# # Combine standardized numeric data with the 'date' column
# standardized_data_firm_t = pd.concat([data[['Ticker' ,'date']], standardized_data], axis=1)

# # Display the standardized DataFrame
# standardized_data_firm_t



In [324]:
# option_test_c  = option_test.copy()

# # Apply filter of moneyness
# option_test_c = option_test_c[option_test_c["cp_flag"] == "C"]
# option_test_c = option_test_c[(option_test_c["moneyness"] >= -0.5) & (option_test_c["moneyness"] <= 2)]

# import pandas as pd

# def standardize(df):
#     """Standardize the numeric columns, excluding the 'date' column."""
#     numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
#     df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
#     return df

# # Assuming your DataFrame is called 'macro_train'
# data = option_test_c.copy()

# # Exclude the 'date' column from the columns to be standardized
# columns_to_standardize = data.columns.difference(['Ticker', 'date', 'cp_flag'])

# # Standardize the entire DataFrame (except 'date')
# standardized_data = standardize(data[columns_to_standardize])

# # Combine standardized numeric data with the 'date' column
# standardized_data_opt_c_t = pd.concat([data[['Ticker' ,'date', 'cp_flag']], standardized_data], axis=1)

# # Display the standardized DataFrame
# standardized_data_opt_c_t


In [325]:
# option_test_p  = option_test.copy()
# option_test_p = option_test_p[option_test_p["cp_flag"] == "P"]
# option_test_p = option_test_p[(option_test_p["moneyness"] >= -2) & (option_test_p["moneyness"] <= 0.5)]

# import pandas as pd

# def standardize(df):
#     """Standardize the numeric columns, excluding the 'date' column."""
#     numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
#     df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
#     return df

# # Assuming your DataFrame is called 'macro_train'
# data = option_test_p.copy()

# # Exclude the 'date' column from the columns to be standardized
# columns_to_standardize = data.columns.difference(['Ticker', 'date', 'cp_flag'])

# # Standardize the entire DataFrame (except 'date')
# standardized_data = standardize(data[columns_to_standardize])

# # Combine standardized numeric data with the 'date' column
# standardized_data_opt_p_t = pd.concat([data[['Ticker' ,'date', 'cp_flag']], standardized_data], axis=1)

# # Display the standardized DataFrame
# standardized_data_opt_p_t


In [326]:
# # Concatenate the two DataFrames vertically
# optiondata_test_scaled = pd.concat([standardized_data_opt_c_t, standardized_data_opt_p_t], axis=0)

# # Reset the index if necessary
# optiondata_test_scaled.reset_index(drop=True, inplace=True)

# optiondata_test_scaled

# merge_data_test_standardized = pd.merge(optiondata_test_scaled, standardized_data_firm_t, on=['date', 'Ticker'], how='left')

# # Merge the data but now with the macro data
# merge_data_test_scaled = pd.merge(merge_data_test_standardized, standardized_data_macro_t, on='date', how='left')

# merge_data_test_scaled

# # Check for NaN values column-wise
# nan_columns = merge_data_test_scaled.isnull().sum()
# print("Number of NaN values in each column:")
# print(nan_columns)

# # Add columns with NaN values to a list
# nan_columns_list = nan_columns[nan_columns > 0].index.tolist()
# print("Columns with NaN values:")
# print(nan_columns_list)

# # # Drop columns with NaN values
# merge_data_test_scaled = merge_data_test_scaled.drop(columns=nan_columns_list)

# nan_columns = merge_data_test_scaled.isnull().sum()
# print("Number of NaN values in each column:")
# print(nan_columns)


In [327]:
# # Save the parquet file

# data_set_test_scaled = Path.cwd().parent.parent / "Data/updated_standardization/data_set_test_tech_scaled.parquet"

# save_parquet = True
# if save_parquet or not os.path.exists(data_set_test_scaled):
#     merge_data_test_scaled.to_parquet(data_set_test_scaled)

# Create validation and testset

In [328]:
option_train_c = option_train2[option_train2["cp_flag"] == "C"]
option_train_c2 = option_train_c[(option_train_c["moneyness"] >= -0.5) & (option_train_c["moneyness"] <= 2)]

option_train_p = option_train2[option_train2["cp_flag"] == "P"]
option_train_p2 = option_train_p[(option_train_p["moneyness"] >= -2) & (option_train_p["moneyness"] <= 0.5)]

# Concatenate the two DataFrames vertically
optiondata_train2 = pd.concat([option_train_c2, option_train_p2], axis=0)

# Reset the index if necessary
optiondata_train2.reset_index(drop=True, inplace=True)

merge_data_train2 = pd.merge(optiondata_train2, firm_train2, on=['date', 'Ticker'], how='left')

# Merge the data but now with the macro data
validation_train2 = pd.merge(merge_data_train2, macro_train2, on='date', how='left')

# Check for NaN values column-wise
nan_columns = validation_train2.isnull().sum()
print("Number of NaN values in each column:")
print(nan_columns)

# Add columns with NaN values to a list
nan_columns_list = nan_columns[nan_columns > 0].index.tolist()
print("Columns with NaN values:")
print(nan_columns_list)

# # Drop columns with NaN values
merge_data_train_c1 = validation_train2.drop(columns=nan_columns_list)

nan_columns = merge_data_train_c1.isnull().sum()
print("Number of NaN values in each column:")
print(nan_columns)

validation_train2


Number of NaN values in each column:
cp_flag                  0
Ticker                   0
date                     0
trading_days_till_exp    0
moneyness                0
                        ..
OPEN_vix                 0
HIGH_vix                 0
LOW_vix                  0
CLOSE_vix                0
spread_vix               0
Length: 126, dtype: int64
Columns with NaN values:
[]
Number of NaN values in each column:
cp_flag                  0
Ticker                   0
date                     0
trading_days_till_exp    0
moneyness                0
                        ..
OPEN_vix                 0
HIGH_vix                 0
LOW_vix                  0
CLOSE_vix                0
spread_vix               0
Length: 126, dtype: int64


,cp_flag,Ticker,date,trading_days_till_exp,moneyness,impl_volatility,T,prev_day_iv,prev2_day_iv,BIDLO,...,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,HIGH_vix,LOW_vix,CLOSE_vix,spread_vix
0,C,AAPL,2020-01-02,1,-0.415,0.395077,0.007937,0.219748,0.257932,285.22,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
1,C,AAPL,2020-01-02,1,-0.133,0.342038,0.007937,0.212392,0.253059,285.22,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
2,C,AAPL,2020-01-02,1,0.146,0.320892,0.007937,0.209724,0.251866,285.22,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
3,C,AAPL,2020-01-02,1,0.423,0.306323,0.007937,0.209129,0.253682,285.22,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
4,C,AAPL,2020-01-02,1,0.698,0.294776,0.007937,0.211117,0.259173,285.22,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87496,P,TSLA,2020-12-31,5,0.236,0.573773,0.023810,0.557458,0.432122,655.00,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
87497,P,TSLA,2020-12-31,5,0.274,0.573565,0.023810,0.543927,0.431956,655.00,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
87498,P,TSLA,2020-12-31,5,0.312,0.572960,0.023810,0.543927,0.431956,655.00,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
87499,P,TSLA,2020-12-31,5,0.388,0.573072,0.023810,0.531722,0.432987,655.00,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26


In [329]:
# Save the parquet file

data_set_val_nonscaled = Path.cwd().parent.parent / "Data/updated_standardization/data_set_val_tech_nonscaled.parquet"

save_parquet = True
if save_parquet or not os.path.exists(data_set_val_nonscaled):
    validation_train2.to_parquet(data_set_val_nonscaled)

# Create a scaled dataset by starting with the train period calls

In [330]:
firm_train1

,Ticker,date,BIDLO,ASKHI,PRC,vol_stock,RET,BID,ASK,RETX,...,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2
0,AAPL,2019-01-02,154.55000,158.52000,156.23000,42291347.0,0.000512,156.22000,156.24001,0.000512,...,0.056201,6.0,0.027847,2.548018e-12,0.040515,0.027893,0.344719,3.369334,1.040351e-08,36.0
1,AAPL,2019-01-03,156.48000,159.36000,157.74001,35003466.0,0.009665,157.92999,157.94000,0.009665,...,0.056201,6.0,0.028037,2.557111e-12,0.041546,0.027900,0.341555,3.361944,1.046380e-08,36.0
2,AAPL,2019-01-04,154.23000,158.85001,157.92000,37066356.0,0.001141,157.91000,157.92999,0.001141,...,0.056201,6.0,0.028228,2.566203e-12,0.042578,0.027907,0.338390,3.354553,1.052410e-08,36.0
3,AAPL,2019-01-07,142.00000,145.72000,142.19000,91373695.0,-0.099607,142.08000,142.09000,-0.099607,...,0.056201,6.0,0.028801,2.593480e-12,0.045671,0.027927,0.328897,3.332380,1.070499e-08,36.0
4,AAPL,2019-01-08,143.80000,148.54990,148.25999,58603001.0,0.042689,148.25000,148.25999,0.042689,...,0.056201,6.0,0.028991,2.602572e-12,0.046703,0.027934,0.325733,3.324990,1.076528e-08,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5546,TSLA,2019-12-24,400.18500,413.00000,405.59000,14750787.0,0.003836,405.48999,405.54999,0.003836,...,0.174166,4.0,0.027660,5.803530e-12,0.061703,0.026152,0.460671,23.352328,2.269810e-09,37.0
5547,TSLA,2019-12-26,410.00000,422.01001,419.22000,13298127.0,0.033605,419.10999,419.17999,0.033605,...,0.174166,4.0,0.027109,5.693842e-12,0.052504,0.024693,0.449543,22.061294,2.334807e-09,37.0
5548,TSLA,2019-12-27,412.68750,425.47000,425.25000,8186207.0,0.014384,425.10001,425.19000,0.014384,...,0.174166,4.0,0.026834,5.638999e-12,0.047905,0.023964,0.443979,21.415777,2.367305e-09,37.0
5549,TSLA,2019-12-30,426.35001,433.48001,430.94000,10617605.0,0.013380,430.82001,430.92001,0.013380,...,0.174166,4.0,0.026008,5.474468e-12,0.034106,0.021775,0.427287,19.479225,2.464799e-09,37.0


In [331]:
# Apply filter of moneyness
option_train_c = option_train1[option_train1["cp_flag"] == "C"]
option_train_c = option_train_c[(option_train_c["moneyness"] >= -0.5) & (option_train_c["moneyness"] <= 2)]

def standardize(df):
    """Standardize the numeric columns, excluding the 'date' column."""
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
    return df

# Assuming your DataFrame is called 'macro_train'
data = option_train_c.copy()

# Exclude the 'date' column from the columns to be standardized
columns_to_standardize = data.columns.difference(['Ticker', 'date', 'cp_flag'])

# Standardize the entire DataFrame (except 'date')
standardized_data = standardize(data[columns_to_standardize])

# Combine standardized numeric data with the 'date' column
standardized_data_opt_c_t1 = pd.concat([data[['Ticker' ,'date', 'cp_flag']], standardized_data], axis=1)

# Display the standardized DataFrame
standardized_data_opt_c_t1


/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_9160/529030889.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()


,Ticker,date,cp_flag,T,impl_volatility,moneyness,prev2_day_iv,prev_day_iv,trading_days_till_exp
64436,AAPL,2019-01-02,C,-0.847372,-0.245084,-1.694497,-1.715833,-1.684428,-0.847372
64437,AAPL,2019-01-02,C,-0.847372,-0.226914,-1.107359,-1.715833,-0.487047,-0.847372
64438,AAPL,2019-01-02,C,-0.847372,-0.236555,-0.528609,-1.715833,-0.422614,-0.847372
64439,AAPL,2019-01-02,C,-0.847372,-0.223718,0.040355,-1.715833,-0.386493,-0.847372
64440,AAPL,2019-01-02,C,-0.847372,-0.202050,0.600932,-1.715833,-0.357108,-0.847372
...,...,...,...,...,...,...,...,...,...
214650,TSLA,2019-12-31,C,-0.847372,0.425400,1.045479,0.644425,0.977332,-0.847372
214651,TSLA,2019-12-31,C,-0.847372,0.470649,1.199253,0.664717,1.029089,-0.847372
214652,TSLA,2019-12-31,C,-0.847372,0.489891,1.350231,0.691227,1.078811,-0.847372
214653,TSLA,2019-12-31,C,-0.847372,0.544302,1.502607,0.720056,1.120815,-0.847372


In [332]:
# Apply filter of moneyness
option_train_p = option_train1[option_train1["cp_flag"] == "P"]
option_train_p = option_train_p[(option_train_p["moneyness"] >= -2) & (option_train_p["moneyness"] <= 0.5)]

import pandas as pd

def standardize(df):
    """Standardize the numeric columns, excluding the 'date' column."""
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
    return df

# Assuming your DataFrame is called 'macro_train'
data = option_train_p.copy()

# Exclude the 'date' column from the columns to be standardized
columns_to_standardize = data.columns.difference(['Ticker', 'date', 'cp_flag'])

# Standardize the entire DataFrame (except 'date')
standardized_data = standardize(data[columns_to_standardize])

# Combine standardized numeric data with the 'date' column
standardized_data_opt_p_t1 = pd.concat([data[['Ticker' ,'date', 'cp_flag']], standardized_data], axis=1)

# Display the standardized DataFrame
standardized_data_opt_p_t1


/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_9160/4007751345.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()


,Ticker,date,cp_flag,T,impl_volatility,moneyness,prev2_day_iv,prev_day_iv,trading_days_till_exp
251194,AAPL,2019-01-02,P,-0.846775,0.421655,-1.568502,-1.733221,0.323252,-0.846775
251195,AAPL,2019-01-02,P,-0.846775,0.359962,-1.319855,-1.733221,0.280146,-0.846775
251196,AAPL,2019-01-02,P,-0.846775,0.300854,-1.072606,-1.733221,0.236354,-0.846775
251197,AAPL,2019-01-02,P,-0.846775,0.236877,-0.828150,-1.733221,0.192603,-0.846775
251198,AAPL,2019-01-02,P,-0.846775,0.095105,-0.220504,-1.733221,0.121395,-0.846775
...,...,...,...,...,...,...,...,...,...
392762,TSLA,2019-12-31,P,-0.846775,0.248954,1.043681,0.258539,0.363153,-0.846775
392763,TSLA,2019-12-31,P,-0.846775,0.269870,1.208514,0.259157,0.386832,-0.846775
392764,TSLA,2019-12-31,P,-0.846775,0.277903,1.373347,0.263376,0.412048,-0.846775
392765,TSLA,2019-12-31,P,-0.846775,0.298151,1.535386,0.280704,0.464358,-0.846775


# Standardize firm characteristics

In [333]:
def standardize(df):
    """Standardize the numeric columns, excluding the 'date' column."""
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
    return df

# Assuming your DataFrame is called 'macro_train'
data = firm_train1.copy()

# Exclude the 'date' column from the columns to be standardized
columns_to_standardize = data.columns.difference(['Ticker', 'date'])

# Standardize the entire DataFrame (except 'date')
standardized_data = standardize(data[columns_to_standardize])

# Combine standardized numeric data with the 'date' column
standardized_data_firm_t1 = pd.concat([data[['Ticker' ,'date']], standardized_data], axis=1)

# Display the standardized DataFrame
standardized_data_firm_t1

/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_9160/1924348484.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()


,Ticker,date,5_day_rolling_return_stock,ASK,ASKHI,BID,BIDLO,PRC,PRC_actual,RET,...,spread_stock,std_dolvol,std_turn,stdacc,stdcf,tang,tb,turn,vol_stock,zerotrade
0,AAPL,2019-01-02,-0.307275,-0.505712,-0.505068,-0.505698,-0.504837,-0.505707,-0.822966,-0.071612,...,-0.426741,0.545217,-0.380475,-1.535547,-1.515170,-1.989614,-0.303392,-0.506294,2.383640,-0.311142
1,AAPL,2019-01-03,0.807128,-0.502869,-0.503673,-0.502837,-0.501580,-0.503181,-0.816294,0.371267,...,-0.479031,0.495464,-0.381395,-1.535547,-1.515170,-1.989614,-0.303392,-0.504821,1.731332,-0.304651
2,AAPL,2019-01-04,1.449699,-0.502885,-0.504520,-0.502871,-0.505377,-0.502880,-0.815499,-0.041177,...,-0.426845,0.445712,-0.382316,-1.535547,-1.515170,-1.989614,-0.303392,-0.503348,1.915973,-0.298160
3,AAPL,2019-01-07,-2.332808,-0.529379,-0.526331,-0.529354,-0.526012,-0.529194,-0.884997,-4.915996,...,-0.479083,0.296454,-0.385077,-1.535547,-1.515170,-1.989614,-0.303392,-0.498929,6.776798,-0.278687
4,AAPL,2019-01-08,-1.341300,-0.519059,-0.521630,-0.519032,-0.522975,-0.519040,-0.858179,1.969175,...,-0.479136,0.246701,-0.385998,-1.535547,-1.515170,-1.989614,-0.303392,-0.497456,3.843629,-0.272196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5546,TSLA,2019-12-24,2.693347,-0.088722,-0.082340,-0.088674,-0.090389,-0.088564,-1.035361,0.089224,...,-0.217633,2.368262,2.108093,1.553935,0.835866,-0.191921,0.382637,1.749715,-0.081401,-1.186759
5547,TSLA,2019-12-26,1.966520,-0.065924,-0.067373,-0.065888,-0.073828,-0.065763,-1.019302,1.529634,...,-0.165343,2.193302,1.947315,1.553935,0.835866,-0.191921,0.382637,1.756744,-0.211423,-1.179762
5548,TSLA,2019-12-27,2.479921,-0.055872,-0.061625,-0.055866,-0.069294,-0.055676,-1.012197,0.599602,...,-0.060815,2.105821,1.866926,1.553935,0.835866,-0.191921,0.382637,1.760259,-0.668970,-1.176263
5549,TSLA,2019-12-30,1.905592,-0.046288,-0.048319,-0.046297,-0.046242,-0.046157,-1.005494,0.551022,...,-0.008472,1.843381,1.625759,1.553935,0.835866,-0.191921,0.382637,1.770802,-0.451345,-1.165767


# Standardized macro data

In [334]:
# Standardize micro data
import pandas as pd

def standardize(df):
    """Standardize the numeric columns, excluding the 'date' column."""
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()
    return df

# Assuming your DataFrame is called 'macro_train'
data = macro_train1.copy()

# Exclude the 'date' column from the columns to be standardized
columns_to_standardize = data.columns.difference(['date'])

# Standardize the entire DataFrame (except 'date')
standardized_data = standardize(data[columns_to_standardize])

# Combine standardized numeric data with the 'date' column
standardized_data_macro_t1 = pd.concat([data[['date']], standardized_data], axis=1)

# Display the standardized DataFrame
standardized_data_macro_t1


/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_9160/942207535.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()


,date,10Y_RIR,1Y_bond,2Y_bond,CLOSE_vix,FF_rate,HIGH_vix,LOW_vix,OPEN_vix,gold_price,reces_indi,spread_vix
3,2019-01-02,1.286663,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
4,2019-01-03,1.286663,1.307506,1.421618,2.267485,0.733069,3.558028,2.698761,4.140757,-1.207092,1.573545,4.063041
5,2019-01-04,1.038280,1.140601,1.198933,3.302776,0.733069,2.975638,3.195899,2.709562,-1.196444,1.592066,1.412664
8,2019-01-07,1.311501,1.263478,1.354732,1.726498,0.733069,1.915271,2.102196,2.257873,-1.185795,1.610586,0.813781
9,2019-01-08,1.386016,1.362758,1.466746,1.896758,0.733069,1.692524,2.183546,2.100543,-1.153850,1.666149,0.030136
...,...,...,...,...,...,...,...,...,...,...,...,...
359,2019-12-24,-0.526532,-1.168736,-0.815977,-1.299736,-1.880068,-1.309917,-1.287382,-1.211000,1.227346,-0.310944,-0.874559
361,2019-12-26,-0.601047,-1.199239,-0.865402,-1.373882,-1.880068,-1.402728,-1.317511,-1.345492,1.247094,-0.310944,-1.065692
362,2019-12-27,-0.601047,-1.197800,-0.871581,-1.269529,-1.880068,-1.384166,-1.251226,-1.276977,1.286590,-0.310944,-1.154888
365,2019-12-30,-0.650724,-1.256505,-0.947868,-0.978439,-1.880068,-1.054687,-1.073462,-1.223688,1.306338,-0.310944,-0.626087


# Combine all the training data

In [335]:
merge_train_c1 = pd.merge(standardized_data_opt_c_t1, standardized_data_firm_t1, on=['date', 'Ticker'], how='left')

# Merge the data but now with the macro data
merge_data_train_c1 = pd.merge(merge_train_c1, standardized_data_macro_t1, on='date', how='left')

merge_data_train_c1

# Check for NaN values column-wise
nan_columns = merge_data_train_c1.isnull().sum()
print("Number of NaN values in each column:")
print(nan_columns)

# Add columns with NaN values to a list
nan_columns_list = nan_columns[nan_columns > 0].index.tolist()
print("Columns with NaN values:")
print(nan_columns_list)

# # Drop columns with NaN values
merge_data_train_c1 = merge_data_train_c1.drop(columns=nan_columns_list)

nan_columns = merge_data_train_c1.isnull().sum()
print("Number of NaN values in each column:")
print(nan_columns)


Number of NaN values in each column:
Ticker             0
date               0
cp_flag            0
T                  0
impl_volatility    0
                  ..
LOW_vix            0
OPEN_vix           0
gold_price         0
reces_indi         0
spread_vix         0
Length: 126, dtype: int64
Columns with NaN values:
[]
Number of NaN values in each column:
Ticker             0
date               0
cp_flag            0
T                  0
impl_volatility    0
                  ..
LOW_vix            0
OPEN_vix           0
gold_price         0
reces_indi         0
spread_vix         0
Length: 126, dtype: int64


In [336]:
merge_train_p1 = pd.merge(standardized_data_opt_p_t1, standardized_data_firm_t1, on=['date', 'Ticker'], how='left')

# Merge the data but now with the macro data
merge_data_train_p1 = pd.merge(merge_train_p1, standardized_data_macro_t1, on='date', how='left')

merge_data_train_p1

# Check for NaN values column-wise
nan_columns = merge_data_train_p1.isnull().sum()
print("Number of NaN values in each column:")
print(nan_columns)

# Add columns with NaN values to a list
nan_columns_list = nan_columns[nan_columns > 0].index.tolist()
print("Columns with NaN values:")
print(nan_columns_list)

# # Drop columns with NaN values
merge_data_train_p1 = merge_data_train_p1.drop(columns=nan_columns_list)

nan_columns = merge_data_train_p1.isnull().sum()
print("Number of NaN values in each column:")
print(nan_columns)
merge_data_train_p1

Number of NaN values in each column:
Ticker             0
date               0
cp_flag            0
T                  0
impl_volatility    0
                  ..
LOW_vix            0
OPEN_vix           0
gold_price         0
reces_indi         0
spread_vix         0
Length: 126, dtype: int64
Columns with NaN values:
[]
Number of NaN values in each column:
Ticker             0
date               0
cp_flag            0
T                  0
impl_volatility    0
                  ..
LOW_vix            0
OPEN_vix           0
gold_price         0
reces_indi         0
spread_vix         0
Length: 126, dtype: int64


,Ticker,date,cp_flag,T,impl_volatility,moneyness,prev2_day_iv,prev_day_iv,trading_days_till_exp,5_day_rolling_return_stock,...,1Y_bond,2Y_bond,CLOSE_vix,FF_rate,HIGH_vix,LOW_vix,OPEN_vix,gold_price,reces_indi,spread_vix
0,AAPL,2019-01-02,P,-0.846775,0.421655,-1.568502,-1.733221,0.323252,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
1,AAPL,2019-01-02,P,-0.846775,0.359962,-1.319855,-1.733221,0.280146,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
2,AAPL,2019-01-02,P,-0.846775,0.300854,-1.072606,-1.733221,0.236354,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
3,AAPL,2019-01-02,P,-0.846775,0.236877,-0.828150,-1.733221,0.192603,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
4,AAPL,2019-01-02,P,-0.846775,0.095105,-0.220504,-1.733221,0.121395,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28636,TSLA,2019-12-31,P,-0.846775,0.248954,1.043681,0.258539,0.363153,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
28637,TSLA,2019-12-31,P,-0.846775,0.269870,1.208514,0.259157,0.386832,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
28638,TSLA,2019-12-31,P,-0.846775,0.277903,1.373347,0.263376,0.412048,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
28639,TSLA,2019-12-31,P,-0.846775,0.298151,1.535386,0.280704,0.464358,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184


In [337]:
merge_data_train_c1

,Ticker,date,cp_flag,T,impl_volatility,moneyness,prev2_day_iv,prev_day_iv,trading_days_till_exp,5_day_rolling_return_stock,...,1Y_bond,2Y_bond,CLOSE_vix,FF_rate,HIGH_vix,LOW_vix,OPEN_vix,gold_price,reces_indi,spread_vix
0,AAPL,2019-01-02,C,-0.847372,-0.245084,-1.694497,-1.715833,-1.684428,-0.847372,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
1,AAPL,2019-01-02,C,-0.847372,-0.226914,-1.107359,-1.715833,-0.487047,-0.847372,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
2,AAPL,2019-01-02,C,-0.847372,-0.236555,-0.528609,-1.715833,-0.422614,-0.847372,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
3,AAPL,2019-01-02,C,-0.847372,-0.223718,0.040355,-1.715833,-0.386493,-0.847372,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
4,AAPL,2019-01-02,C,-0.847372,-0.202050,0.600932,-1.715833,-0.357108,-0.847372,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29164,TSLA,2019-12-31,C,-0.847372,0.425400,1.045479,0.644425,0.977332,-0.847372,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
29165,TSLA,2019-12-31,C,-0.847372,0.470649,1.199253,0.664717,1.029089,-0.847372,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
29166,TSLA,2019-12-31,C,-0.847372,0.489891,1.350231,0.691227,1.078811,-0.847372,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
29167,TSLA,2019-12-31,C,-0.847372,0.544302,1.502607,0.720056,1.120815,-0.847372,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184


In [338]:
# Save the parquet file

data_set_train_p = Path.cwd().parent.parent / "Data/updated_standardization/data_set_train_tech_scaled_p.parquet"
data_set_train_c = Path.cwd().parent.parent / "Data/updated_standardization/data_set_train_tech_scaled_c.parquet"

save_parquet = True
if save_parquet or not os.path.exists(data_set_train_p):
    merge_data_train_p1.to_parquet(data_set_train_p)

save_parquet = True
if save_parquet or not os.path.exists(data_set_train_c):
    merge_data_train_c1.to_parquet(data_set_train_c)

In [339]:
validation_c = validation_train2[validation_train2["cp_flag"] == "C"]
validation_p = validation_train2[validation_train2["cp_flag"] == "P"]
test_c = merge_data_total_t[merge_data_total_t["cp_flag"] == "C"]
test_p = merge_data_total_t[merge_data_total_t["cp_flag"] == "P"]

# Example custom standardization function (as you might have implemented)
def custom_standardize(data, means=None, stds=None):
    if means is None or stds is None:
        means = data.mean()
        stds = data.std()
    standardized_data = (data - means) / stds
    return standardized_data, means, stds

# Step 1: Extract features and target from train and test data
train_features = merge_data_train_c1.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
validate_features_c = validation_c.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])

# Step 2: Standardize the training data and save the means and stds
train_features_standardized, train_means, train_stds = custom_standardize(train_features)

# # Step 3: Standardize the test data using the saved means and stds from the training data
validatefeatures_c_standardized, _, _ = custom_standardize(validate_features_c, means=train_means, stds=train_stds)

# Step 4: Combine back the standardized test features with the non-feature columns if needed
val_data_c_standardized = pd.DataFrame(validatefeatures_c_standardized, columns=validate_features_c.columns)
val_data_c_standardized = pd.concat([validation_c[['date', 'Ticker', 'cp_flag', 'impl_volatility']], val_data_c_standardized], axis=1)

val_data_c_standardized



,date,Ticker,cp_flag,impl_volatility,trading_days_till_exp,moneyness,T,prev_day_iv,prev2_day_iv,BIDLO,...,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,HIGH_vix,LOW_vix,CLOSE_vix,spread_vix
0,2020-01-02,AAPL,C,0.395077,1.0,-0.415,0.007937,0.219748,0.257932,214.778000,...,537.805668,-0.079156,1.795788,1.528833,1.522701,13.801291,14.084911,12.449324,12.649523,1.460534
1,2020-01-02,AAPL,C,0.342038,1.0,-0.133,0.007937,0.212392,0.253059,214.778000,...,537.805668,-0.079156,1.795788,1.528833,1.522701,13.801291,14.084911,12.449324,12.649523,1.460534
2,2020-01-02,AAPL,C,0.320892,1.0,0.146,0.007937,0.209724,0.251866,214.778000,...,537.805668,-0.079156,1.795788,1.528833,1.522701,13.801291,14.084911,12.449324,12.649523,1.460534
3,2020-01-02,AAPL,C,0.306323,1.0,0.423,0.007937,0.209129,0.253682,214.778000,...,537.805668,-0.079156,1.795788,1.528833,1.522701,13.801291,14.084911,12.449324,12.649523,1.460534
4,2020-01-02,AAPL,C,0.294776,1.0,0.698,0.007937,0.211117,0.259173,214.778000,...,537.805668,-0.079156,1.795788,1.528833,1.522701,13.801291,14.084911,12.449324,12.649523,1.460534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43173,2020-12-31,TSLA,C,0.623205,5.0,1.467,0.023810,0.554273,0.486818,494.178651,...,673.389773,2.753459,0.823569,0.016081,0.048386,20.563300,20.973907,19.517599,20.395473,1.195664
43174,2020-12-31,TSLA,C,0.629719,5.0,1.535,0.023810,0.554273,0.486818,494.178651,...,673.389773,2.753459,0.823569,0.016081,0.048386,20.563300,20.973907,19.517599,20.395473,1.195664
43175,2020-12-31,TSLA,C,0.639880,5.0,1.670,0.023810,0.584788,0.511182,494.178651,...,673.389773,2.753459,0.823569,0.016081,0.048386,20.563300,20.973907,19.517599,20.395473,1.195664
43176,2020-12-31,TSLA,C,0.648805,5.0,1.804,0.023810,0.601543,0.523462,494.178651,...,673.389773,2.753459,0.823569,0.016081,0.048386,20.563300,20.973907,19.517599,20.395473,1.195664


In [340]:
# Save the parquet file
data_set_val_scaled_c = Path.cwd().parent.parent / "Data/updated_standardization/data_set_val_tech_scaled_c.parquet"

save_parquet = True
if save_parquet or not os.path.exists(data_set_val_scaled_c):
    val_data_c_standardized.to_parquet(data_set_val_scaled_c)

In [341]:
merge_data_train_p1

,Ticker,date,cp_flag,T,impl_volatility,moneyness,prev2_day_iv,prev_day_iv,trading_days_till_exp,5_day_rolling_return_stock,...,1Y_bond,2Y_bond,CLOSE_vix,FF_rate,HIGH_vix,LOW_vix,OPEN_vix,gold_price,reces_indi,spread_vix
0,AAPL,2019-01-02,P,-0.846775,0.421655,-1.568502,-1.733221,0.323252,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
1,AAPL,2019-01-02,P,-0.846775,0.359962,-1.319855,-1.733221,0.280146,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
2,AAPL,2019-01-02,P,-0.846775,0.300854,-1.072606,-1.733221,0.236354,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
3,AAPL,2019-01-02,P,-0.846775,0.236877,-0.828150,-1.733221,0.192603,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
4,AAPL,2019-01-02,P,-0.846775,0.095105,-0.220504,-1.733221,0.121395,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28636,TSLA,2019-12-31,P,-0.846775,0.248954,1.043681,0.258539,0.363153,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
28637,TSLA,2019-12-31,P,-0.846775,0.269870,1.208514,0.259157,0.386832,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
28638,TSLA,2019-12-31,P,-0.846775,0.277903,1.373347,0.263376,0.412048,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
28639,TSLA,2019-12-31,P,-0.846775,0.298151,1.535386,0.280704,0.464358,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184


In [342]:
# Filter the validation set for 'C' and 'P' flags
validation_c = validation_train2[validation_train2["cp_flag"] == "C"]
validation_p = validation_train2[validation_train2["cp_flag"] == "P"]

# Custom standardization function
def custom_standardize(data, means=None, stds=None):
    if means is None or stds is None:
        means = data.mean()
        stds = data.std()
    standardized_data = (data - means) / stds
    return standardized_data, means, stds

# Step 1: Extract features from train data and the 'P' validation data
train_features = merge_data_train_p1.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
validate_features_p = validation_p.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])

# Step 2: Standardize the training data and save the means and stds
train_features_standardized, train_means, train_stds = custom_standardize(train_features)

# Step 3: Standardize the 'P' validation data using the saved means and stds from the training data
validate_features_p_standardized, _, _ = custom_standardize(validate_features_p, means=train_means, stds=train_stds)

# Step 4: Combine back the standardized 'P' validation features with the non-feature columns
val_data_p_standardized = pd.DataFrame(validate_features_p_standardized, columns=validate_features_p.columns)
val_data_p_standardized = pd.concat([validation_p[['date', 'Ticker', 'cp_flag', 'impl_volatility']], val_data_p_standardized], axis=1)

# Output the standardized 'P' validation data
val_data_p_standardized


,date,Ticker,cp_flag,impl_volatility,trading_days_till_exp,moneyness,T,prev_day_iv,prev2_day_iv,BIDLO,...,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,HIGH_vix,LOW_vix,CLOSE_vix,spread_vix
43178,2020-01-02,AAPL,P,0.627576,1.0,-1.861,0.007937,0.316222,0.297076,213.557684,...,539.617660,-0.099719,1.780072,1.516304,1.506719,13.811509,14.086645,12.440672,12.641558,1.465170
43179,2020-01-02,AAPL,P,0.589063,1.0,-1.566,0.007937,0.306290,0.326549,213.557684,...,539.617660,-0.099719,1.780072,1.516304,1.506719,13.811509,14.086645,12.440672,12.641558,1.465170
43180,2020-01-02,AAPL,P,0.512918,1.0,-1.275,0.007937,0.284871,0.307900,213.557684,...,539.617660,-0.099719,1.780072,1.516304,1.506719,13.811509,14.086645,12.440672,12.641558,1.465170
43181,2020-01-02,AAPL,P,0.488522,1.0,-0.985,0.007937,0.265837,0.291318,213.557684,...,539.617660,-0.099719,1.780072,1.516304,1.506719,13.811509,14.086645,12.440672,12.641558,1.465170
43182,2020-01-02,AAPL,P,0.429982,1.0,-0.699,0.007937,0.249253,0.279259,213.557684,...,539.617660,-0.099719,1.780072,1.516304,1.506719,13.811509,14.086645,12.440672,12.641558,1.465170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87496,2020-12-31,TSLA,P,0.573773,5.0,0.236,0.023810,0.557458,0.432122,491.325769,...,675.656277,2.712515,0.809390,0.004844,0.036547,20.582290,20.979565,19.508384,20.386063,1.199299
87497,2020-12-31,TSLA,P,0.573565,5.0,0.274,0.023810,0.543927,0.431956,491.325769,...,675.656277,2.712515,0.809390,0.004844,0.036547,20.582290,20.979565,19.508384,20.386063,1.199299
87498,2020-12-31,TSLA,P,0.572960,5.0,0.312,0.023810,0.543927,0.431956,491.325769,...,675.656277,2.712515,0.809390,0.004844,0.036547,20.582290,20.979565,19.508384,20.386063,1.199299
87499,2020-12-31,TSLA,P,0.573072,5.0,0.388,0.023810,0.531722,0.432987,491.325769,...,675.656277,2.712515,0.809390,0.004844,0.036547,20.582290,20.979565,19.508384,20.386063,1.199299


In [343]:
# Save the parquet file
data_set_val_scaled_p = Path.cwd().parent.parent / "Data/updated_standardization/data_set_val_tech_scaled_p.parquet"

save_parquet = True
if save_parquet or not os.path.exists(data_set_val_scaled_p):
    val_data_p_standardized.to_parquet(data_set_val_scaled_p)

In [344]:
import pandas as pd

# Assuming merge_data_total_t and merge_data_train_c1 are already defined

# Filter the test set for 'C' and 'P' flags
test_c = merge_data_total_t[merge_data_total_t["cp_flag"] == "C"]
test_p = merge_data_total_t[merge_data_total_t["cp_flag"] == "P"]

# Custom standardization function
def custom_standardize(data, means=None, stds=None):
    if means is None or stds is None:
        means = data.mean()
        stds = data.std()
    standardized_data = (data - means) / stds
    return standardized_data, means, stds

# Step 1: Extract features from train data and the 'C' test data
train_features = merge_data_train_c1.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
test_features_c = test_c.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])

# Step 2: Standardize the training data and save the means and stds
train_features_standardized, train_means, train_stds = custom_standardize(train_features)

# Step 3: Standardize the 'C' test data using the saved means and stds from the training data
test_features_c_standardized, _, _ = custom_standardize(test_features_c, means=train_means, stds=train_stds)

# Step 4: Combine back the standardized 'C' test features with the non-feature columns
test_data_c_standardized = pd.DataFrame(test_features_c_standardized, columns=test_features_c.columns)
test_data_c_standardized = pd.concat([test_c[['date', 'Ticker', 'cp_flag', 'impl_volatility']], test_data_c_standardized], axis=1)

# Output the standardized 'C' test data
test_data_c_standardized



,date,Ticker,cp_flag,impl_volatility,trading_days_till_exp,moneyness,T,prev_day_iv,prev2_day_iv,BIDLO,...,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,HIGH_vix,LOW_vix,CLOSE_vix,spread_vix
0,2021-01-04,AAPL,C,0.433412,4.0,-0.475,0.019841,0.337114,0.325650,100.064902,...,674.581266,2.747453,0.823569,0.013005,0.044187,20.361305,21.621260,19.527150,20.289627,1.843125
1,2021-01-04,AAPL,C,0.428775,4.0,-0.345,0.019841,0.332640,0.312515,100.064902,...,674.581266,2.747453,0.823569,0.013005,0.044187,20.361305,21.621260,19.527150,20.289627,1.843125
2,2021-01-04,AAPL,C,0.426358,4.0,-0.216,0.019841,0.330864,0.308907,100.064902,...,674.581266,2.747453,0.823569,0.013005,0.044187,20.361305,21.621260,19.527150,20.289627,1.843125
3,2021-01-04,AAPL,C,0.422649,4.0,-0.088,0.019841,0.329431,0.306933,100.064902,...,674.581266,2.747453,0.823569,0.013005,0.044187,20.361305,21.621260,19.527150,20.289627,1.843125
4,2021-01-04,AAPL,C,0.421479,4.0,0.039,0.019841,0.328318,0.305940,100.064902,...,674.581266,2.747453,0.823569,0.013005,0.044187,20.361305,21.621260,19.527150,20.289627,1.843125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36347,2021-12-30,TSLA,C,0.742669,1.0,1.556,0.007937,0.718058,0.773147,814.108973,...,635.297620,-0.271641,1.432434,0.390523,0.706535,13.695484,13.997953,11.876221,12.255009,1.960845
36348,2021-12-30,TSLA,C,0.764381,1.0,1.658,0.007937,0.735985,0.789361,814.108973,...,635.297620,-0.271641,1.432434,0.390523,0.706535,13.695484,13.997953,11.876221,12.255009,1.960845
36349,2021-12-30,TSLA,C,0.786317,1.0,1.760,0.007937,0.754168,0.807648,814.108973,...,635.297620,-0.271641,1.432434,0.390523,0.706535,13.695484,13.997953,11.876221,12.255009,1.960845
36350,2021-12-30,TSLA,C,0.798978,1.0,1.861,0.007937,0.773210,0.821686,814.108973,...,635.297620,-0.271641,1.432434,0.390523,0.706535,13.695484,13.997953,11.876221,12.255009,1.960845


In [345]:
# Save the parquet file
data_set_val_test_c = Path.cwd().parent.parent / "Data/updated_standardization/data_set_test_tech_scaled_c.parquet"

save_parquet = True
if save_parquet or not os.path.exists(data_set_val_test_c):
    test_data_c_standardized.to_parquet(data_set_val_test_c)

In [346]:
merge_data_train_p1

,Ticker,date,cp_flag,T,impl_volatility,moneyness,prev2_day_iv,prev_day_iv,trading_days_till_exp,5_day_rolling_return_stock,...,1Y_bond,2Y_bond,CLOSE_vix,FF_rate,HIGH_vix,LOW_vix,OPEN_vix,gold_price,reces_indi,spread_vix
0,AAPL,2019-01-02,P,-0.846775,0.421655,-1.568502,-1.733221,0.323252,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
1,AAPL,2019-01-02,P,-0.846775,0.359962,-1.319855,-1.733221,0.280146,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
2,AAPL,2019-01-02,P,-0.846775,0.300854,-1.072606,-1.733221,0.236354,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
3,AAPL,2019-01-02,P,-0.846775,0.236877,-0.828150,-1.733221,0.192603,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
4,AAPL,2019-01-02,P,-0.846775,0.095105,-0.220504,-1.733221,0.121395,-0.846775,-0.307275,...,1.307506,1.421618,3.846509,0.733069,3.622996,4.431212,4.206734,-1.217741,1.555024,0.578051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28636,TSLA,2019-12-31,P,-0.846775,0.248954,1.043681,0.258539,0.363153,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
28637,TSLA,2019-12-31,P,-0.846775,0.269870,1.208514,0.259157,0.386832,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
28638,TSLA,2019-12-31,P,-0.846775,0.277903,1.373347,0.263376,0.412048,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184
28639,TSLA,2019-12-31,P,-0.846775,0.298151,1.535386,0.280704,0.464358,-0.846775,1.220814,...,-1.270893,-0.968821,-0.217761,-1.880068,-0.386446,-0.865568,-1.053670,1.365583,-0.310944,0.769184


In [347]:
import pandas as pd

# Assuming merge_data_total_t and merge_data_train_c1 are already defined

# Filter the test set for 'C' and 'P' flags
test_c = merge_data_total_t[merge_data_total_t["cp_flag"] == "C"]
test_p = merge_data_total_t[merge_data_total_t["cp_flag"] == "P"]

# Custom standardization function
def custom_standardize(data, means=None, stds=None):
    if means is None or stds is None:
        means = data.mean()
        stds = data.std()
    standardized_data = (data - means) / stds
    return standardized_data, means, stds

# Step 1: Extract features from train data and the 'C' test data
train_features = merge_data_train_p1.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
test_features_p = test_p.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])

# Step 2: Standardize the training data and save the means and stds
train_features_standardized, train_means, train_stds = custom_standardize(train_features)

# Step 3: Standardize the 'C' test data using the saved means and stds from the training data
test_features_p_standardized, _, _ = custom_standardize(test_features_p, means=train_means, stds=train_stds)

# Step 4: Combine back the standardized 'C' test features with the non-feature columns
test_data_p_standardized = pd.DataFrame(test_features_p_standardized, columns=test_features_p.columns)
test_data_p_standardized = pd.concat([test_p[['date', 'Ticker', 'cp_flag', 'impl_volatility']], test_data_p_standardized], axis=1)

# Output the standardized 'C' test data
test_data_p_standardized



,date,Ticker,cp_flag,impl_volatility,trading_days_till_exp,moneyness,T,prev_day_iv,prev2_day_iv,BIDLO,...,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,HIGH_vix,LOW_vix,CLOSE_vix,spread_vix
36352,2021-01-04,AAPL,P,0.622737,4.0,-1.977,0.019841,0.404451,0.452074,99.514866,...,676.851764,2.706552,0.809390,0.001770,0.032360,20.380034,21.627287,19.517935,20.280237,1.849205
36353,2021-01-04,AAPL,P,0.598160,4.0,-1.835,0.019841,0.383928,0.418943,99.514866,...,676.851764,2.706552,0.809390,0.001770,0.032360,20.380034,21.627287,19.517935,20.280237,1.849205
36354,2021-01-04,AAPL,P,0.570362,4.0,-1.694,0.019841,0.374351,0.403192,99.514866,...,676.851764,2.706552,0.809390,0.001770,0.032360,20.380034,21.627287,19.517935,20.280237,1.849205
36355,2021-01-04,AAPL,P,0.549241,4.0,-1.554,0.019841,0.365382,0.388048,99.514866,...,676.851764,2.706552,0.809390,0.001770,0.032360,20.380034,21.627287,19.517935,20.280237,1.849205
36356,2021-01-04,AAPL,P,0.526871,4.0,-1.415,0.019841,0.349066,0.360382,99.514866,...,676.851764,2.706552,0.809390,0.001770,0.032360,20.380034,21.627287,19.517935,20.280237,1.849205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71690,2021-12-30,TSLA,P,0.484270,1.0,-0.027,0.007937,0.520039,0.592762,809.386706,...,637.436430,-0.290819,1.417292,0.378966,0.692846,13.705565,13.999638,11.867614,12.247118,1.967370
71691,2021-12-30,TSLA,P,0.503165,1.0,0.081,0.007937,0.520221,0.600908,809.386706,...,637.436430,-0.290819,1.417292,0.378966,0.692846,13.705565,13.999638,11.867614,12.247118,1.967370
71692,2021-12-30,TSLA,P,0.536047,1.0,0.190,0.007937,0.525230,0.609799,809.386706,...,637.436430,-0.290819,1.417292,0.378966,0.692846,13.705565,13.999638,11.867614,12.247118,1.967370
71693,2021-12-30,TSLA,P,0.550971,1.0,0.298,0.007937,0.529985,0.620276,809.386706,...,637.436430,-0.290819,1.417292,0.378966,0.692846,13.705565,13.999638,11.867614,12.247118,1.967370


In [348]:
# Save the parquet file
data_set_val_test_p = Path.cwd().parent.parent / "Data/updated_standardization/data_set_test_tech_scaled_p.parquet"

save_parquet = True
if save_parquet or not os.path.exists(data_set_val_test_p):
    test_data_p_standardized.to_parquet(data_set_val_test_p)